# Import Library and Data

In [ ]:
!pip install urllib3
import urllib3
import json
import numpy as np
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Masking, Dense, Embedding, Reshape

Using TensorFlow backend.


In [ ]:
http = urllib3.PoolManager()

request = http.request('GET', 'http://abc.sourceforge.net/NMD/nmd/jigs.txt')

data = request.data

In [ ]:
spliter_list = ['||', ':|', '|:', '::', '|', '[', ']', '!segno!', '!trill!'] 

def custom_split(text, index_spliter): 
  num_text = len(text)
  spliter = spliter_list[index_spliter]

  len_spliter = len(spliter) 
  spread_list = [ text[i : i + len_spliter] for i in range(num_text - len_spliter + 1) ] 

  spliter_without_self = spliter_list.copy()
  spliter_without_self.remove(spliter)
  spliter_index = [i for i, m in enumerate(spread_list) if m == spliter]

  if len(spliter_index) > 0 and text not in spliter_without_self: 
    melodys = []

    melodys.append(text[0: spliter_index[0]]) 

    for i in range(len(spliter_index) - 1): 
       spliter_i = spliter_index[i]
       spliter_j = spliter_index[i + 1]
      #  melodys.append(text[spliter_i: spliter_i+len_spliter]) 
       melodys.append(text[spliter_i+len_spliter: spliter_j]) 

    # melodys.append(text[spliter_index[-1]: spliter_index[-1]+len_spliter])   
    melodys.append(text[spliter_index[-1]+len_spliter: num_text]) 

  else:
    melodys = [text]  

  new_melodys = []
  for melody in melodys: 
    new_melody = melody 
    
    if new_melody != '' and new_melody[0].isnumeric():
      new_melody = new_melody[1:]

    if new_melody != '' and '"' in new_melody:
      splited = new_melody.split('"')
      new_melody = splited[0] + splited[-1] 

    new_melody = ''.join([m for m in new_melody if m != '\\'])

    if len(new_melody) > 1: 
      new_melodys.append(new_melody)
      
  if index_spliter == len(spliter_list) - 1:
    return new_melodys
  else: 
    return [m for melody in new_melodys for m in custom_split(melody, index_spliter + 1)]

In [ ]:
header = ['X:', 'T:', '% ', '%%', '%P', 'S:', 'M:', 'K:', 'P:', 'L:', 'V:']
delete_melody = ['']

musics = []

for d in str(data)[2:-1].split(r'\n\n\n'): 
  music = []
  for i, s in enumerate(d.split(r'\n')):
    if s[:2] not in header:  
      melody = [m for melody in r'{:}'.format(s).split() for m in custom_split(melody, 0) if m not in delete_melody]
      music += melody
  musics.append(music)

In [ ]:
index_to_melody = sorted(list(set([melody for music in musics for melody in music])) + ['STOP'])
melody_to_index = {m:i for i, m in enumerate(index_to_melody)} 

n_unique = len(index_to_melody)
melody_to_one_hot = {} 

for i in range(n_unique): 
  one_hot = np.zeros(n_unique) 
  one_hot[i] = 1 
  melody_to_one_hot[index_to_melody[i]] = one_hot 

In [ ]:
special_value = -n_unique
max_seq_len = max([len(music) for music in musics])

In [ ]:
data_melody = pd.DataFrame(columns=['X', 'y', 'X_index', 'y_one_hot', 'length'])

for music in musics:
  for i in range(1, len(music) + 1):
    X = music[0:i]
    length = len(X)
    X_one_hot = np.full(max_seq_len, special_value)
    X_one_hot[0:length] = [melody_to_index[melody] for melody in X]
    y = music[i] if i != len(music) else 'STOP'
    y_one_hot = melody_to_one_hot[y]
    data_melody = data_melody.append([{'X': X, 'X_index': X_one_hot, 'y':y, 'y_one_hot': y_one_hot, 'length': length}], ignore_index=True)
    
data_melody.head()

,X,y,X_index,y_one_hot,length
0,[ecc],c2f,"[1064, -1242, -1242, -1242, -1242, -1242, -124...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
1,"[ecc, c2f]",ecc,"[1064, 817, -1242, -1242, -1242, -1242, -1242,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2
2,"[ecc, c2f, ecc]",c2f,"[1064, 817, 1064, -1242, -1242, -1242, -1242, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3
3,"[ecc, c2f, ecc, c2f]",ecc,"[1064, 817, 1064, 817, -1242, -1242, -1242, -1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4
4,"[ecc, c2f, ecc, c2f, ecc]",c2f,"[1064, 817, 1064, 817, 1064, -1242, -1242, -12...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5


# Create Model

In [ ]:
def create_model(): 
  model = Sequential()
 
  model.add(Masking(mask_value=special_value, input_shape=(max_seq_len, 1))) 

  model.add(Embedding(input_dim = n_unique, output_dim = 256)) 

  model.add(Reshape((max_seq_len, 256)))

  model.add(LSTM(128, return_sequences = True))

  model.add(LSTM(128))

  model.add(Dense(n_unique, activation='softmax'))

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ["accuracy"]) 

  return model

In [ ]:
model = create_model()

# model = load_model('path/gen_melody_20_epochs.h5')

model.summary()






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 127, 1)            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 127, 1, 256)       317952    
_________________________________________________________________
reshape_1 (Reshape)          (None, 127, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 127, 128)          197120    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1242)              160218    
Total params: 806,874
Trainable params: 806,874
Non-trainable params: 0
___________________________________________

In [ ]:
X_index = data_melody['X_index']

X = np.array(list(X_index.values)).reshape([X_index.shape[0], max_seq_len, 1]) 
y = np.array(list(data_melody['y_one_hot'].values))

model.fit(X, y, batch_size=64, epochs=10)

Epoch 1/10
12911/12911 [==============================] - 130s 10ms/step - loss: 5.9767 - acc: 0.0267
Epoch 2/10
12911/12911 [==============================] - 130s 10ms/step - loss: 5.9721 - acc: 0.0246
Epoch 3/10
12911/12911 [==============================] - 131s 10ms/step - loss: 5.9618 - acc: 0.0263
Epoch 4/10
12911/12911 [==============================] - 130s 10ms/step - loss: 5.9636 - acc: 0.0257
Epoch 5/10
12911/12911 [==============================] - 130s 10ms/step - loss: 5.9567 - acc: 0.0265
Epoch 6/10
12911/12911 [==============================] - 130s 10ms/step - loss: 5.9572 - acc: 0.0249
Epoch 7/10
12911/12911 [==============================] - 130s 10ms/step - loss: 5.9573 - acc: 0.0252
Epoch 8/10
12911/12911 [==============================] - 130s 10ms/step - loss: 5.9530 - acc: 0.0238
Epoch 9/10
12911/12911 [==============================] - 130s 10ms/step - loss: 5.9573 - acc: 0.0250
Epoch 10/10
12911/12911 [==============================] - 130s 10ms/step - loss: 

In [ ]:
#model.save('path/model_melody_1.h5')

# Generate Melody 

In [ ]:
first_melody_list = data_melody[data_melody.length == 1] 
first_melody_list['X'] = first_melody_list['X'].apply(lambda x: x[0])
dict_weight = first_melody_list[['X', 'length']].groupby(['X']).count().to_dict()['length'] 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
def roulette_wheel(items, prob):
  prob_normal = np.array(prob) / sum(prob)
  return np.random.choice(a=items, size=1, p=prob_normal)[0] 

def generate_music():
  first_melody = roulette_wheel(list(dict_weight.keys()), list(dict_weight.values())) 

  melody = [first_melody]
  index = 0

  X = np.full((1, max_seq_len, 1), special_value)
  X[0, index, 0] = melody_to_index[first_melody]

  while True:
    y_pred = model.predict(X)
    next_melody = roulette_wheel(index_to_melody, y_pred[0])
    if next_melody == 'STOP' or len(melody) > max_seq_len : 
      break 
    else: 
      if next_melody[0] != '-' or index % 2 == 0 :
        melody.append(next_melody)
        index += 1 
        X[0, index, 0] = melody_to_index[first_melody]
  
  if len(melody) % 2 != 0 :
    melody = melody[:-1]
  return melody 

def print_melody(melodys): 
  lines = len(melodys) // 10 
  outer = len(melodys) % 10 
  for i in range(lines): 
    pairs = [' '.join([melodys[i*10 + j], melodys[i*10 + j + 1]]) for j in range(0, 10, 2)] 
    notes = '|'.join(pairs) + '|' 
    print(notes)  
  pairs = [' '.join([melodys[lines*10 + j], melodys[lines*10 + j + 1]]) for j in range(0, outer, 2)] 
  notes = '|'.join(pairs) + '||' 
  print(notes) 

In [ ]:
music = generate_music() 

print_melody(music)

d2^c ABc|fed Agg|cBc e3|E2e gag|FAd dcA|
A2 c2e|A2 D-FA|B^AB cBA|de^f fdd|d2d Ace|
gdB A2G|g2g d3|FGA e3||
